# Data Cleaning / Wrangling

**Goal** : gather all the data in one clean csv file


## Step 1 : get consumption

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df1 = pd.read_csv("./data/eco2mix_regional_cons_def.csv", delimiter=";")
df1["Date - Heure"] = pd.to_datetime(df1["Date - Heure"],utc=True)

In [ ]:
df1.head(1)

In [ ]:
consumption = df1[["Date - Heure","Consommation (MW)"]].copy().sort_values(by=['Date - Heure'])
consumption["Date - Heure"] = consumption["Date - Heure"].dt.tz_convert("Europe/Paris")

In [ ]:
consumption.head()

In [ ]:
print(consumption["Date - Heure"].min())
print(consumption["Date - Heure"].max())

In [ ]:
# Count duplicates
# Identical dates
print(consumption.duplicated(subset='Date - Heure').sum())
# Identical dates and conso
print(consumption.duplicated().sum())

We have duplicated dates with different consumptions, **interesting** !

In [ ]:
# Remove duplicates
consumption.drop_duplicates(inplace=True, subset='Date - Heure')

In [ ]:
# Rename columns
consumption.columns = ['Date', 'Conso']

In [ ]:
# Check days with missing half hours
count_half_hours = consumption.set_index('Date').resample('D').count()

In [ ]:
count_half_hours[count_half_hours['Conso'] != 48]

We have days with less that 48 half hours, **interesting** !

## Step 3 : get all half hours

In [ ]:
date_range = pd.date_range(start=consumption['Date'].min(),end=consumption['Date'].max(),freq='30min')
half_hours = pd.DataFrame(date_range,columns=['Date'])

In [ ]:
half_hours

## Step 2 : get temperatures

In [ ]:
df2 = pd.read_csv("./data/meteo-paris.csv")

In [ ]:
df2.head(1)

In [ ]:
weather = df2[['dt','temp']].copy()
weather.columns = ['Date', 'Temp']

In [ ]:
weather.head(3)

In [ ]:
# Count duplicates
weather.duplicated().sum()

In [ ]:
# Remove duplicates
weather.drop_duplicates(inplace=True,subset='Date')

In [ ]:
weather['Date'] = pd.to_datetime(weather['Date'],unit='s',utc=True).dt.tz_convert('Europe/Paris')

In [ ]:
weather.head()

In [ ]:
consumption.head()

## Step 3 : Merge everything together

[Documentation on how to merge with pandas](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html)


![How to merge](https://shanelynnwebsite-mid9n9g1q9y8tt.netdna-ssl.com/wp-content/uploads/2017/03/join-types-merge-names.jpg)


**Inner Merge / Inner join** – The default Pandas behaviour, only keep rows where the merge “on” value exists in both the left and right dataframes.

**Left Merge / Left outer join** – Keep every row in the left dataframe. Where there are missing values of the “on” variable in the right dataframe, add empty / NaN values in the result.

**Right Merge / Right outer join** – Keep every row in the right dataframe. Where there are missing values of the “on” variable in the left column, add empty / NaN values in the result.

**Outer Merge / Full outer join** – A full outer join returns all the rows from the left dataframe, all the rows from the right dataframe, and matches up rows where possible, with NaNs elsewhere.

### Merge consumption and weather

In [ ]:
print("Consumption shape",consumption.shape)
print("Weather shape",weather.shape)

leftmerge = pd.merge(consumption,weather,on='Date',how="left")
innermerge = pd.merge(consumption,weather,on='Date',how="inner")

print("Left merge", leftmerge.shape)
print("Inner merge", innermerge.shape)

### Merge with dates

In [ ]:
df = pd.merge(half_hours,leftmerge,on='Date',how="left")

In [ ]:
df.shape

In [ ]:
df.head(100)

In [ ]:
df.set_index('Date',inplace=True)
df.sort_index(inplace=True)

In [ ]:
df.head()

# Step 4 : interpolate missing temperatures
We want to keep our historical consumptions (which are precious) so we will interpolate missing values for temperature

First question : where are missing values ?

In [ ]:
df.interpolate('linear',limit=4,inplace=True)

In [ ]:
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
weather['Date'].max()

In [ ]:
df[:'2017-12-05'].isnull().sum()

In [ ]:
df.head(10)

# Step 5 : Automate everything : 

In [ ]:
def get_data(consumption_csv="./data/eco2mix_regional_cons_def.csv",weather_csv="./data/meteo-paris.csv"):
    """
    A function to get consumption and weather data
    Do the wrangling
    And return a nice & compact dataframe
    Temperatures are in °C
    
    """
    # consumptions
    consumption =  pd.read_csv(consumption_csv, delimiter=";", usecols = ["Date - Heure","Consommation (MW)"])
    consumption["Date - Heure"] = pd.to_datetime(consumption["Date - Heure"],utc=True)
    consumption["Date - Heure"] = consumption["Date - Heure"].dt.tz_convert("Europe/Paris")
    consumption.drop_duplicates(inplace=True,subset='Date - Heure')
    consumption.columns = ['Date', 'Conso']
    # half hours
    date_range = pd.date_range(start=consumption['Date'].min(),end=consumption['Date'].max(),freq='30min')
    half_hours = pd.DataFrame(date_range,columns=['Date'])
    # weather
    weather = pd.read_csv(weather_csv,usecols=['dt','temp'])
    weather.columns = ['Date', 'Temp']
    weather.drop_duplicates(inplace=True,subset='Date')
    weather['Date'] = pd.to_datetime(weather['Date'],unit='s',utc=True).dt.tz_convert('Europe/Paris')
    # Merging
    df1 = pd.merge(consumption,weather,on='Date',how="left")
    df1["Temp"] = df1["Temp"] - 273.15
    
    date_range = pd.date_range(start=df1['Date'].min(),end=df1['Date'].max(),freq='30min')
    half_hours = pd.DataFrame(date_range,columns=['Date'])
    df2 = pd.merge(half_hours,df1,on='Date',how="left")
    df2.interpolate('linear',limit=4,inplace=True)
    return df2.dropna()

In [ ]:
df = get_data()


In [ ]:
test = df2.set_index('Date').resample('D').count()

In [ ]:
test[test['Conso']!=48]

In [ ]:
df.index.max()

In [ ]:
df.isnull().sum()

In [ ]:
data.to_csv("mydata.csv")

# That's Clean !